In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import math

In [18]:
# Read in text data

reddit_data = pd.read_csv("/Users/eugen/Desktop/Work/BrainStation/Suicide Project/Suicide_Detection.csv", index_col = 0).reset_index(drop=True) #with open()
reddit_data.head()

,text,class
0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,Am I weird I don't get affected by compliments...,non-suicide
2,Finally 2020 is almost over... So I can never ...,non-suicide
3,i need helpjust help me im crying so hard,suicide
4,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


## 1. Data Pre-processing and Cleaning

In [19]:
print(f"Dataset has {reddit_data.shape[0]} and {reddit_data.shape[1]} columns")

Dataset has 232074 and 2 columns


In [20]:
reddit_data.info() #no missing values and both columns are in the right format

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232074 entries, 0 to 232073
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    232074 non-null  object
 1   class   232074 non-null  object
dtypes: object(2)
memory usage: 3.5+ MB


In [21]:
reddit_data.text.unique()

array(["Ex Wife Threatening SuicideRecently I left my wife for good because she has cheated on me twice and lied to me so much that I have decided to refuse to go back to her. As of a few days ago, she began threatening suicide. I have tirelessly spent these paat few days talking her out of it and she keeps hesitating because she wants to believe I'll come back. I know a lot of people will threaten this in order to get their way, but what happens if she really does? What do I do and how am I supposed to handle her death on my hands? I still love my wife but I cannot deal with getting cheated on again and constantly feeling insecure. I'm worried today may be the day she does it and I hope so much it doesn't happen.",
       "Am I weird I don't get affected by compliments if it's coming from someone I know irl but I feel really good when internet strangers do it",
       'Finally 2020 is almost over... So I can never hear "2020 has been a bad year" ever again. I swear to fucking God it\'

The text column contains reddit posts for the users; however, it is very messy. We need to thoroughly clean each text because our analysis and model performance will depend on it. 

In the above snippet, we have:
- Contractions: e.g. I'm
- Puncuations: e.g. ", !
- Emojis
- Numbers.

In the part below, we are going to expand contractions, lower case all letters, remove punctuations

In [ ]:
#lowercase all text before expanding contractions

### 1.1. Expand contractions

In [ ]:
# dictionary of common English contractions and their full meaning
contractions_dict = {"ain't": "are not","'s":" is","aren't": "are not", "can't": "cannot","can't've": "cannot have",
"'cause": "because","could've": "could have","couldn't": "could not", "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
"don't": "do not","hadn't": "had not","hadn't've": "had not have", "hasn't": "has not","haven't": "have not","he'd": "he would",
"he'd've": "he would have","he'll": "he will", "he'll've": "he will have", "how'd": "how did","how'd'y": "how do you","how'll": "how will",
"I'd": "I would", "I'd've": "I would have","I'll": "I will", "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
"it'd": "it would","it'd've": "it would have","it'll": "it will", "it'll've": "it will have", "let's": "let us","ma'am": "madam",
"mayn't": "may not","might've": "might have","mightn't": "might not", "mightn't've": "might not have","must've": "must have","mustn't": "must not",
"mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
"oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not", "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
"she'll": "she will", "she'll've": "she will have","should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
"that'd": "that would","that'd've": "that would have", "there'd": "there would", "there'd've": "there would have", "they'd": "they would",
"they'd've": "they would have","they'll": "they will", "they'll've": "they will have", "they're": "they are","they've": "they have",
"to've": "to have","wasn't": "was not","we'd": "we would", "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
"we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will", "what'll've": "what will have","what're": "what are", "what've": "what have",
"when've": "when have","where'd": "where did", "where've": "where have", "who'll": "who will","who'll've": "who will have","who've": "who have",
"why've": "why have","will've": "will have","won't": "will not", "won't've": "will not have", "would've": "would have","wouldn't": "would not",
"wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have","y'all're": "you all are",
"y'all've": "you all have", "you'd": "you would","you'd've": "you would have", "you'll": "you will","you'll've": "you will have", "you're": "you are",
"you've": "you have"}

Footnote: contraction dictionary from [Analytics Vidya](https://www.analyticsvidhya.com/blog/2020/04/beginners-guide-exploratory-data-analysis-text-data/)

In [ ]:
#finding the contractions in our dataset's text
pattern = r""" (?x)
        


"""